In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [69]:
# Read : Please avoid to read what i made, I am just a begginer 
# Read : I am just practicing here in kaggle because my own computer system is unable to take load and hung.


import pandas as pd
credits  = pd.read_csv("../input/the-movies-dataset/credits.csv")
credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [ ]:
keywords = pd.read_csv("../input/the-movies-dataset/keywords.csv")
keywords.columns

In [ ]:
metadata = pd.read_csv("../input/the-movies-dataset/movies_metadata.csv")
print(metadata.columns)
metadata = metadata.drop([19730, 29503, 35587])

In [ ]:
# Id is common: we can use Id to merge the data
print(metadata.dtypes) # id is object
print("-----------------------------------")
print(credits.dtypes) # id is int
print("-----------------------------------")
print(keywords.dtypes) #id is int

In [ ]:
# converting all id type to int for precaution
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

In [ ]:
# merge datasets metadata will now have to merge with k and c based on id
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [ ]:
print(metadata.columns) 
# output : cast Crew and keyword are added
print(metadata.dtypes)

In [ ]:
# Parse the stringified features into recommended
from ast import literal_eval
#----------------------------------------------------------------------


features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [ ]:
import numpy as np

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [ ]:
# Print the new features of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

In [ ]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [ ]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)

In [ ]:
metadata[['soup']].head(2)

In [ ]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

In [ ]:
count_matrix.shape

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix[1:4000], count_matrix[1:4000])

# because of memory error i am making only short metrix 

In [ ]:
# Now recommendation function
# Function that recommends movies as per its title.

def get_recommendations(title, cosine_sim = cosine_sim2):
    #index of movie
    idx = indices[title]
    
    #pairwise similarity with given movie with other movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #sort movie based on similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #score of 7 most similar movie
    sim_scores = sim_scores[1:11]
    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [ ]:
# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['title'])

In [ ]:
get_recommendations('Big Shots', cosine_sim2)
# Here also i can not give movie name whoes index is bigger than 4k.

In [ ]:
# user based collaborative  filtering :  recommend products to a user that similar users have liked
# item based collaborative  filtering : Based on similar item
